In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/pubmed-multilabel-text-classification/PubMed Multi Label Text Classification Dataset Processed.csv
/kaggle/input/pubmed-multilabel-text-classification/PubMed Multi Label Text Classification Dataset.csv


In [2]:
file_path = '/kaggle/input/pubmed-multilabel-text-classification/PubMed Multi Label Text Classification Dataset Processed.csv'
data = pd.read_csv(file_path)

print("Размер данных:", data.shape)
print("\nПервые 5 строк:")
data.head()

Размер данных: (50000, 20)

Первые 5 строк:


,Title,abstractText,meshMajor,pmid,meshid,meshroot,A,B,C,D,E,F,G,H,I,J,L,M,N,Z
0,Expression of p53 and coexistence of HPV in pr...,Fifty-four paraffin embedded tissue sections f...,"['DNA Probes, HPV', 'DNA, Viral', 'Female', 'H...",8549602,"[['D13.444.600.223.555', 'D27.505.259.750.600....","['Chemicals and Drugs [D]', 'Organisms [B]', '...",0,1,1,1,1,0,0,1,0,0,0,0,0,0
1,Vitamin D status in pregnant Indian women acro...,The present cross-sectional study was conducte...,"['Adult', 'Alkaline Phosphatase', 'Breast Feed...",21736816,"[['M01.060.116'], ['D08.811.277.352.650.035'],...","['Named Groups [M]', 'Chemicals and Drugs [D]'...",0,1,1,1,1,1,1,0,1,1,0,1,1,1
2,[Identification of a functionally important di...,The occurrence of individual amino acids and d...,"['Amino Acid Sequence', 'Analgesics, Opioid', ...",19060934,"[['G02.111.570.060', 'L01.453.245.667.060'], [...","['Phenomena and Processes [G]', 'Information S...",1,1,0,1,1,0,1,0,0,0,1,0,0,0
3,Multilayer capsules: a promising microencapsul...,"In 1980, Lim and Sun introduced a microcapsule...","['Acrylic Resins', 'Alginates', 'Animals', 'Bi...",11426874,"[['D05.750.716.822.111', 'D25.720.716.822.111'...","['Chemicals and Drugs [D]', 'Technology, Indus...",1,1,1,1,1,0,1,0,0,1,0,0,0,0
4,"Nanohydrogel with N,N'-bis(acryloyl)cystine cr...",Substantially improved hydrogel particles base...,"['Antineoplastic Agents', 'Cell Proliferation'...",28323099,"[['D27.505.954.248'], ['G04.161.750', 'G07.345...","['Chemicals and Drugs [D]', 'Phenomena and Pro...",1,1,0,1,1,0,1,0,0,1,0,0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Title         49998 non-null  object
 1   abstractText  50000 non-null  object
 2   meshMajor     50000 non-null  object
 3   pmid          50000 non-null  int64 
 4   meshid        50000 non-null  object
 5   meshroot      50000 non-null  object
 6   A             50000 non-null  int64 
 7   B             50000 non-null  int64 
 8   C             50000 non-null  int64 
 9   D             50000 non-null  int64 
 10  E             50000 non-null  int64 
 11  F             50000 non-null  int64 
 12  G             50000 non-null  int64 
 13  H             50000 non-null  int64 
 14  I             50000 non-null  int64 
 15  J             50000 non-null  int64 
 16  L             50000 non-null  int64 
 17  M             50000 non-null  int64 
 18  N             50000 non-null  int64 
 19  Z   

In [4]:
missing_values = data.isnull().sum()
missing_percentage = (data.isnull().sum() / len(data)) * 100

missing_info = pd.DataFrame({
    'Пропущенные значения': missing_values,
    'Процент пропусков': missing_percentage
})

print("Пропущенные значения:")
print(missing_info[missing_info['Пропущенные значения'] > 0])

Пропущенные значения:
       Пропущенные значения  Процент пропусков
Title                     2              0.004


Как видно, в датасете практически отсутствуют нулевые значения. За исключением двух строк, у которых отсутствуют заголовки. Удалим эти строки.

In [5]:
data_cleaned = data.dropna()

print(f"Было строк: {len(data)}")
print(f"Стало строк: {len(data_cleaned)}")
print(f"Удалено строк: {len(data) - len(data_cleaned)}")

Было строк: 50000
Стало строк: 49998
Удалено строк: 2


Теперь выполним анализ текста

In [6]:
import re

def simple_normalize_text(text):
    """
    Упрощенная нормализация текста без NLTK
    """
    if not isinstance(text, str):
        return ""
    
    # Базовые стоп-слова
    stop_words = {'the', 'and', 'of', 'to', 'in', 'a', 'an', 'is', 'are', 'was', 'were', 
                 'for', 'on', 'with', 'by', 'at', 'from', 'as', 'this', 'that', 'these', 'those'}
    
    # Приведение к нижнему регистру
    text = text.lower()
    
    # Удаление специальных символов и цифр
    #text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Разделение на слова и фильтрация
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words and len(word) > 2]
    
    return ' '.join(filtered_words)

# Быстрая нормализация
data['simple_title'] = data['Title'].apply(simple_normalize_text)
data['simple_abstract'] = data['abstractText'].apply(simple_normalize_text)

# Покажем разницу
print("Сравнение методов нормализации:")
for i in range(2):
    print(f"\nПример {i+1}:")
    print(f"Оригинал: {data['Title'].iloc[i][:80]}...")
    print(f"Простая: {data['simple_title'].iloc[i][:80]}...")

Сравнение методов нормализации:

Пример 1:
Оригинал: Expression of p53 and coexistence of HPV in premalignant lesions and in cervical...
Простая: expression p53 coexistence hpv premalignant lesions cervical cancer....

Пример 2:
Оригинал: Vitamin D status in pregnant Indian women across trimesters and different season...
Простая: vitamin status pregnant indian women across trimesters different seasons its cor...


In [7]:
data.head()

,Title,abstractText,meshMajor,pmid,meshid,meshroot,A,B,C,D,...,G,H,I,J,L,M,N,Z,simple_title,simple_abstract
0,Expression of p53 and coexistence of HPV in pr...,Fifty-four paraffin embedded tissue sections f...,"['DNA Probes, HPV', 'DNA, Viral', 'Female', 'H...",8549602,"[['D13.444.600.223.555', 'D27.505.259.750.600....","['Chemicals and Drugs [D]', 'Organisms [B]', '...",0,1,1,1,...,0,1,0,0,0,0,0,0,expression p53 coexistence hpv premalignant le...,fifty-four paraffin embedded tissue sections p...
1,Vitamin D status in pregnant Indian women acro...,The present cross-sectional study was conducte...,"['Adult', 'Alkaline Phosphatase', 'Breast Feed...",21736816,"[['M01.060.116'], ['D08.811.277.352.650.035'],...","['Named Groups [M]', 'Chemicals and Drugs [D]'...",0,1,1,1,...,1,0,1,1,0,1,1,1,vitamin status pregnant indian women across tr...,present cross-sectional study conducted determ...
2,[Identification of a functionally important di...,The occurrence of individual amino acids and d...,"['Amino Acid Sequence', 'Analgesics, Opioid', ...",19060934,"[['G02.111.570.060', 'L01.453.245.667.060'], [...","['Phenomena and Processes [G]', 'Information S...",1,1,0,1,...,1,0,0,0,1,0,0,0,[identification functionally important dipepti...,occurrence individual amino acids dipeptide fr...
3,Multilayer capsules: a promising microencapsul...,"In 1980, Lim and Sun introduced a microcapsule...","['Acrylic Resins', 'Alginates', 'Animals', 'Bi...",11426874,"[['D05.750.716.822.111', 'D25.720.716.822.111'...","['Chemicals and Drugs [D]', 'Technology, Indus...",1,1,1,1,...,1,0,0,1,0,0,0,0,multilayer capsules: promising microencapsulat...,"1980, lim sun introduced microcapsule coated a..."
4,"Nanohydrogel with N,N'-bis(acryloyl)cystine cr...",Substantially improved hydrogel particles base...,"['Antineoplastic Agents', 'Cell Proliferation'...",28323099,"[['D27.505.954.248'], ['G04.161.750', 'G07.345...","['Chemicals and Drugs [D]', 'Phenomena and Pro...",1,1,0,1,...,1,0,0,1,0,0,0,0,"nanohydrogel n,n'-bis(acryloyl)cystine crossli...",substantially improved hydrogel particles base...


Генерация TF-IDF признаков из нормализованного текста

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# TF-IDF для заголовков и аннотаций
title_vectorizer = TfidfVectorizer(max_features=5000)
abstract_vectorizer = TfidfVectorizer(max_features=10000)

X_title = title_vectorizer.fit_transform(data['simple_title'])
X_abstract = abstract_vectorizer.fit_transform(data['simple_abstract'])

# Объединение признаков
X = hstack([X_title, X_abstract])

print("\n=== ТОП-10 СЛОВ В ЗАГОЛОВКАХ ===")
feature_names = title_vectorizer.get_feature_names_out()
idf_scores = title_vectorizer.idf_
top_indices = idf_scores.argsort()[:10]  # Самые частые слова
for idx in top_indices:
    print(f"{feature_names[idx]}: IDF={idf_scores[idx]:.3f}")

print("\n=== ТОП-10 РЕДКИХ СЛОВ В ЗАГОЛОВКАХ ===")
top_indices = idf_scores.argsort()[-10:]  # Самые редкие слова
for idx in top_indices:
    print(f"{feature_names[idx]}: IDF={idf_scores[idx]:.3f}")


=== ТОП-10 СЛОВ В ЗАГОЛОВКАХ ===
patients: IDF=3.734
study: IDF=3.828
human: IDF=4.094
cells: IDF=4.115
cell: IDF=4.163
analysis: IDF=4.250
effects: IDF=4.291
cancer: IDF=4.324
effect: IDF=4.355
treatment: IDF=4.382

=== ТОП-10 РЕДКИХ СЛОВ В ЗАГОЛОВКАХ ===
peg: IDF=9.181
amphotericin: IDF=9.181
sugars: IDF=9.181
detachment: IDF=9.181
phenylalanine: IDF=9.181
yl: IDF=9.181
cannabis: IDF=9.181
music: IDF=9.255
psa: IDF=9.255
db: IDF=9.422


Отбор признаков

In [9]:
from sklearn.feature_selection import SelectKBest, chi2


y = data[['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'Z']]
y_target = y['A']

# Отбор top-K наиболее информативных признаков
k_best = SelectKBest(chi2, k=10000)
X_selected = k_best.fit_transform(X, y_target)

print(f"После отбора: {X_selected.shape[1]} признаков для целевой метки A")

После отбора: 10000 признаков для целевой метки A


Разбиение данных на train/OOS/OOT

In [10]:
from sklearn.model_selection import train_test_split

X_temp, X_oot, y_temp, y_oot = train_test_split(X_selected, y, test_size=0.2, shuffle=False)

X_train, X_oos, y_train, y_oos = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

print(f"Train: {X_train.shape}, OOS: {X_oos.shape}, OOT: {X_oot.shape}")

Train: (30000, 10000), OOS: (10000, 10000), OOT: (10000, 10000)


 Обучение логистической регрессии

In [11]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Создаем multi-label модель
model = OneVsRestClassifier(
    LogisticRegression(max_iter=1000, random_state=42, n_jobs=-1)
)

model.fit(X_train, y_train)

# Предсказания
for name, X_val, y_val in [('OOS', X_oos, y_oos), ('OOT', X_oot, y_oot)]:
    y_pred = model.predict(X_val)
    print(f"\n{name} результаты:")
    print(classification_report(y_val, y_pred))


OOS результаты:
              precision    recall  f1-score   support

           0       0.81      0.77      0.79      4583
           1       0.95      0.99      0.97      9270
           2       0.88      0.83      0.85      5269
           3       0.89      0.89      0.89      6111
           4       0.81      0.96      0.88      7842
           5       0.86      0.60      0.71      1797
           6       0.82      0.89      0.85      6680
           7       0.61      0.13      0.21      1195
           8       0.75      0.44      0.56      1133
           9       0.74      0.25      0.37      1104
          10       0.74      0.35      0.47      1507
          11       0.87      0.87      0.87      4246
          12       0.81      0.75      0.78      4609
          13       0.79      0.57      0.66      1612

   micro avg       0.85      0.82      0.84     56958
   macro avg       0.81      0.66      0.71     56958
weighted avg       0.85      0.82      0.82     56958
 samples 

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Модель логистической регрессии демонстрирует высокую эффективность для multi-label классификации медицинских статей с общим Micro F1-score 0.84. Результаты свидетельствует о стабильности модели и отсутствии временного дрейфа данных.